# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
        
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra implementation of project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance to local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute('''Create Keyspace if not exists music_app
    With Replication = 
    {'class':'SimpleStrategy', 'replication_factor':1}''')
    
except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
session.set_keyspace('music_app')

## Queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
##Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#Primary key: Partition Key - sessionId and Clustering Column - itemInSession 
#derived from the WHERE clause conditions — sessionId and itemInSessio. 
#Together, these two columns uniquely identify each record in the dataset.


try:
    query = "Create table if not exists song_library"
    query = query + "(session_id int, item_in_session int, artist varchar, song_title varchar, song_length float, Primary Key(session_id, item_in_session))"
    session.execute(query)
except Exception as e:
    print(e)

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "Insert into song_library (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " Values (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### A SELECT to verify that the data have been inserted into each table

In [ ]:
try:
    query = "Select artist, song_title, song_length from song_library where session_id = 338 and item_in_session = 4"
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#Primary key: Partition Key - user_id and Clustering Columns - session_id and itemInSession 
#derived from the WHERE clause conditions — user_id and session_id aren't unique, 
#so itemInSession is added to make each record unique and help with sorting.


try:
    query = "Create table if not exists user_library"
    query = query + "(user_id int, session_id int, item_in_session int, artist varchar, song_title varchar, firstname varchar, lastname varchar, Primary Key(user_id, session_id, item_in_session))"
    session.execute(query)
except Exception as e:
    print(e)           

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "Insert into user_library (user_id, session_id, item_in_session, artist, song_title, firstname, lastname)"
        query = query + " Values (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1].lower(), line[4].lower()))

In [ ]:
try:
    query = "Select artist, song_title, firstName, lastName, item_in_session from user_library where user_id = 10 and session_id = 182"
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On sylvie cruz
Three Drives Greece 2000 sylvie cruz
Sebastien Tellier Kilometer sylvie cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) sylvie cruz


In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Primary key = (Partition Key: song_title, Clustering Column: user_id) 
# derived from WHERE clause since song_title isn't unique, 
# user_id added to ensure uniqueness and together they uniquely identify each record.
try:
    query = "Create table if not exists user_history"
    query = query + "(song_title varchar, firstname varchar, lastname varchar, user_id int, Primary Key(song_title, user_id))"
    session.execute(query)
except Exception as e:
    print(e)     
                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "Insert into user_history (song_title, firstname, lastname, user_id)"
        query = query + " Values (%s, %s, %s, %s)"
       
        session.execute(query, (line[9], line[1].lower(), line[4].lower(), int(line[10])))
        

In [ ]:
try:
    query = "Select firstName, lastName from user_history where song_title = 'All Hands Against His Own'"
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)


jacqueline lynch
tegan levine
sara johnson


### Drop the tables before closing out the sessions

In [ ]:
query = "Drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
query = "Drop table user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
query = "Drop table user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()